In [42]:
import numpy as np
from PIL import Image
from PIL import ImageStat
from pathlib import Path
import urllib.request
import zipfile
import random

import torch
import torch.utils.data as data
from torchvision import transforms
import csv
import pprint
import math

print(torch.__version__)  # 1.3.1

#urllib.request.urlretrieve(
#    "https://download.pytorch.org/tutorial/hymenoptera_data.zip", 
#    "hymenoptera_data.zip"
#)

#with zipfile.ZipFile("./hymenoptera_data.zip") as zip:
#    zip.extractall(".")
    
#print(list(Path("hymenoptera_data/val").glob("**/*.jpg"))[:4])
# [PosixPath('hymenoptera_data/val/bees/2478216347_535c8fe6d7.jpg'),
#  PosixPath('hymenoptera_data/val/bees/1486120850_490388f84b.jpg'),
#  PosixPath('hymenoptera_data/val/bees/2060668999_e11edb10d0.jpg'),
#  PosixPath('hymenoptera_data/val/bees/2104135106_a65eede1de.jpg')]

1.5.0


In [50]:
class MyDataset(data.Dataset):
    def __init__(self, dir_path, input_size, phase):
        super().__init__()
        
        self.dir_path = dir_path
        self.input_size = input_size
        
        # hymenoptera_data/{train or val}/{ants or bees}/ファイル名.jpg
        # ex. hymenoptera_data/val/bees/2478216347_535c8fe6d7.jpg
        self.image_paths = [str(p) for p in Path(self.dir_path).glob("image/*.png")]
        self.label_paths = [str(q) for q in Path(self.dir_path).glob("label/*.csv")]
        self.len = len(self.image_paths)
        
        transform_ops = []
        if phase == "train":
            transform_ops.append(
                transforms.Resize(input_size)# リサイズ
            )
            #transform_ops.extend([
                #transforms.RandomVerticalFlip(),  # ランダムに左右反転
                #transforms.RandomRotation(20),  # ±20度でランダムに回転
                #transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0))  # ランダムにクロップしてリサイズ
            #])
        elif phase == "val":
            transform_ops.append(
                transforms.Resize(input_size)  # リサイズ
            )
            
        transform_ops.extend([
            transforms.ToTensor(),  # Tensor化
            #transforms.Normalize([0.5], [1.0])  # 標準化
        ])
        
        print(transform_ops)
        
        # 1つの変換処理に集約
        self.transformer = transforms.Compose(transform_ops)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        p = self.image_paths[index]
        q = self.label_paths[index]
        
        # 入力
        #print(p)
        image = Image.open(p)
        image = image.convert('F')
        #image.show()
        
        #max-min標準化
        image_min = -1
        for y in range(image.size[1]):
            for x in range(image.size[0]):
                if image_min == -1:
                    image_min = image.getpixel((x, y))
                elif image.getpixel((x, y)) != 0 and image.getpixel((x, y)) < image_min:
                    image_min = image.getpixel((x, y))
        image_max = image.getextrema()[1]
        print(image_max)
        print(image_min)
        #標準化
        for y in range(image.size[1]):
            for x in range(image.size[0]):
                image.putpixel((x, y), (image.getpixel((x, y)) - image_min) / (image_max - image_min))
        
        
#        #標準化のための平均と標準偏差計算
#        image_sum = 0.
#        image_sqsum = 0.
#        image_num = image.size[0] * image.size[1]
#        for y in range(image.size[1]):
#            for x in range(image.size[0]):
#                image_sum += image.getpixel((x, y))
#                image_sqsum += image.getpixel((x, y)) ** 2
#        image_mean = image_sum / image_num
#        image_sqmean = image_sqsum / image_num
#        image_std = math.sqrt(image_sqmean - (image_mean ** 2))
#        #print(image_mean)
#        #print(image_sqmean)
#        #print(image_std)
#        #標準化
#        for y in range(image.size[1]):
#            for x in range(image.size[0]):
#                image.putpixel((x, y), (image.getpixel((x, y)) - image_mean) / image_std)
       
        #print(image)
        image = self.transformer(image)  # __init__で定義した関数に置き換え
        
        # ラベル (0: ants, 1: bees)
        with open(q) as f:
            reader = csv.reader(f)
            #label = p.split("/")[2]
            label = []
            for row in reader:
                for point in row:
                    label.append(int(point))
        
        return image, label

train_dataset = MyDataset("./monitoring_data/train/", (96, 96), "train")
test_dataset = MyDataset("./monitoring_data/val/", (96, 96), "val")

image, label = train_dataset[0]
print(image.size(), label)  # torch.Size([3, 224, 224]) 1
image

[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), ToTensor()]
[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), ToTensor()]
3868.0
2918.0
torch.Size([1, 224, 224]) [-585, -25, 211, -566, -22, 218, -553, 4, 258, -551, 28, 276, -572, 40, 376, -570, 43, 241, -10000, -10000, -10000, -10000, -10000, -10000, -10000, -10000, -10000, -593, 41, 280, -665, 33, 614, -10000, -10000, -10000, -577, 58, 335, -605, 37, 285, -678, 26, 608, -592, 53, 320, -581, 49, 302, -10000, -10000, -10000, -10000, -10000, -10000, -587, 50, 266, -579, 44, 259]


tensor([[[0.9116, 0.9146, 0.9236,  ..., 0.5379, 0.5379, 0.5379],
         [0.9116, 0.9146, 0.9236,  ..., 0.5379, 0.5379, 0.5379],
         [0.9116, 0.9146, 0.9236,  ..., 0.5379, 0.5379, 0.5379],
         ...,
         [0.3077, 0.3076, 0.3074,  ..., 0.0648, 0.0648, 0.0648],
         [0.3063, 0.3063, 0.3063,  ..., 0.0653, 0.0653, 0.0653],
         [0.3063, 0.3063, 0.3063,  ..., 0.0653, 0.0653, 0.0653]]])

In [ ]:
train_dataloader = data.DataLoader(
    train_dataset, batch_size=16, shuffle=True,
    num_workers=2, drop_last=True
)

test_dataloader = data.DataLoader(
    test_dataset, batch_size=16, shuffle=False,
    num_workers=2, drop_last=True
)

train_images, train_labels = next(iter(train_dataloader))
test_images, test_labels = next(iter(test_dataloader))
print(train_images.size())  # torch.Size([16, 3, 224, 224])
print(train_labels.size())  # torch.Size([16])

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, stride = 2)
        
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 60)
        
    def forward(self, x):
        x = self.conv1(x) #94×94 16chunnel
        x = self.relu(x)
        x = self.pool(x)  #47×47 16chunnel
        x = self.conv2(x) #45×45 32chunnel
        x = self.relu(x)
        x = self.pool(x)  #
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

import torch.optim as optim

device = torch.device("cuda:0")
net = Net()
net = net.to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 0.005)

for epoch in range(100):
    for(inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    for (inputs, labels) in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)